In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from mvf_bto.data_loading import load_data
from mvf_bto.constants import * 
from mvf_bto.models.baseline_lstm import BaselineLSTM
from mvf_bto.preprocessing import create_discharge_inputs
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go

## Loading Data

In [ ]:
# data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"
data_path = "/home/amalss/Documents/CMU/Courses/10701/project/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"


In [ ]:
data = load_data(file_path=data_path, num_cells=3)

## Preprocessing to create model inputs and targets

In [ ]:
train_split = 0.7
test_split = 0.2
# by default uses validation_split = 1 - (train_split + test_split)


In [ ]:
data.keys()

In [ ]:
datasets = create_discharge_inputs(data, train_split, test_split, forecast_horizon=3, history_window=6)

## Train Model

In [ ]:
window_length = datasets["X_train"].shape[1]
n_features = datasets["X_train"].shape[2]
batch_input_shape = (datasets["batch_size"], window_length, n_features)
n_outputs = datasets["y_train"].shape[-1]
nf_steps = datasets["y_train"].shape[1]

y = datasets["y_train"][:, 0, 0]
# print(y)
idx = y < 2.9
weights = np.ones_like(y) * 1
weights[idx] = 2



In [ ]:
model = BaselineLSTM(batch_input_shape=batch_input_shape, n_outputs=n_outputs, nf_steps=nf_steps)

In [ ]:
model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])

es = EarlyStopping(
    monitor="val_mean_squared_error",
    min_delta=0,
    patience=10,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

history = model.fit(
    datasets["X_train"],
    datasets["y_train"],
    validation_data=(datasets["X_val"], datasets["y_val"]),
    epochs=250,
    batch_size=datasets["batch_size"],
    shuffle=False,
    callbacks=[es],
    verbose=1,
    sample_weight=weights
)

In [ ]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.linspace(1, 50),
        y=history.history["loss"],
        showlegend=False,
        mode="markers+lines",
    )
)
fig.update_xaxes(title="Epochs")
fig.update_yaxes(title="Loss (MSE)")
fig.update_layout(height=500)

## Parity Plot of Training Error

In [ ]:
# random plotting traing error at some interval = skip to not make the plot rendering too slow
batch_size = datasets["batch_size"]
skip = 70

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, :, 0])
    df_train = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, :, 0])

    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

    #t+1 predictions
    # fig.add_trace(
    #     go.Scatter(
    #         x=df_pred[1].values,
    #         y=df_train[1].values,
    #         showlegend=False,
    #         mode="markers+lines",
    #     )
    # )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")
fig.update_layout(height=500)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, :, 1])
    df_train = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, :, 1])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Temperature Target")
fig.update_xaxes(title="Normalized Temperature Prediction")
fig.update_layout(height=500)

In [ ]:
skip = 20

pallete = plotly.colors.qualitative.Dark24*100

fig = go.Figure()
for i in range(0, len(datasets["X_train"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_train"][i : i + batch_size][:, 0, 0])
    for j in range(nf_steps):
        df_pred = pd.DataFrame(model.predict(datasets["X_train"][i : i + batch_size], verbose=0)[:, j, 0])
        # df_pred_t2 = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, 1, 0])
        # print(datasets["X_train"][i])
        fig.add_trace(
            go.Scatter(
                x=REFERENCE_CAPACITIES[window_length:-nf_steps],
                y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predicted Curve t %d {i+1}, %f"%(j, datasets["X_train"][i][0, -1]),
                marker_color=pallete[int((1.5*j)*(i//skip))]
            )
        )
       
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CAPACITIES[window_length:-nf_steps],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines+markers",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")
fig.update_layout(height=500)

## Parity Plot of Test Error

In [ ]:
skip = 20

fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, :, 0])
    df_train = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, :, 0])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")
fig.update_layout(height=500)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="markers+lines"))
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, :, 1])
    df_train = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, :, 1])
    fig.add_trace(
        go.Scatter(
            x=df_pred[0].values,
            y=df_train[0].values,
            showlegend=False,
            mode="markers+lines",
        )
    )

fig.update_yaxes(title="Normalized Voltage Target")
fig.update_xaxes(title="Normalized Voltage Prediction")
fig.update_layout(height=500)

## True vs Predicted Traces (Test Set)

In [ ]:
skip = 10

pallete = plotly.colors.qualitative.Dark24*100

fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, 0, 0])
    for j in range(nf_steps):
        df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, j, 0])
        x = model.predict(datasets["X_test"][i : i + batch_size], verbose=0)
        fig.add_trace(
            go.Scatter(
                x=REFERENCE_CAPACITIES[window_length:-nf_steps],
                y=df_pred[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predicted Curve t %d {i+1}"%j,
                marker_color=pallete[int((1.)*(i//skip))]
            )
        )
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CAPACITIES[window_length:-nf_steps],
            y=df_true[0].values*(VOLTAGE_MAX - VOLTAGE_MIN) + VOLTAGE_MIN,
            showlegend=True,
            mode="lines+markers",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Voltage [V]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")
fig.update_layout(height=500)

In [ ]:
fig = go.Figure()
for i in range(0, len(datasets["X_test"]), batch_size * skip):
    df_true = pd.DataFrame(datasets["y_test"][i : i + batch_size][:, 0, 1])
    for j in range(nf_steps):
        df_pred = pd.DataFrame(model.predict(datasets["X_test"][i : i + batch_size], verbose=0)[:, j, 1])
        fig.add_trace(
            go.Scatter(
                x=REFERENCE_CAPACITIES[window_length:-nf_steps],
                y=df_pred[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
                showlegend=True,
                mode="markers",
                name = f"Predicted Curve t%d {i+1}"%j,
                marker_color=pallete[int((1.0)*(i//skip))]
            )
        )
    
    fig.add_trace(
        go.Scatter(
            x=REFERENCE_CAPACITIES[window_length:-nf_steps],
            y=df_true[0].values*(TEMPERATURE_MAX - TEMPERATURE_MIN) + TEMPERATURE_MIN,
            showlegend=True,
            mode="lines+markers",
            name = f"True Curve {i+1}",
            line_color=pallete[i//skip]
        )
    )

fig.update_yaxes(title="Temperature [°C]")
fig.update_xaxes(title="State of Charge (Normalized Capacity)")
fig.update_layout(height=500)